In [ ]:
import torch
from torch import nn
from torch.nn import init

In [ ]:

"""
1. 怎么配置不可训练参数，不参与backforwardl。
2. 复用连接层，共享参数。
3. 加入控制流。
"""
class FancyMLP(nn.Module):
    def __init__(self, **kwargs):
        super(FancyMLP, self).__init__(**kwargs)
        self.rand_weight = torch.rand((20, 20), requires_grad=False) # 不可训练参数（常数参数）
        self.linear = nn.Linear(20, 20)

    def forward(self, x):
        x = self.linear(x)
        # 使用创建的常数参数，以及nn.functional中的relu函数和mm函数
        x = nn.functional.relu(torch.mm(x, self.rand_weight.data) + 1)
        # 复用全连接层。等价于两个全连接层共享参数
        x = self.linear(x)
        # 控制流，这里我们需要调用item函数来返回标量进行比较
        while x.norm().item() > 1:
            x /= 2
        if x.norm().item() < 0.8:
            x *= 10
        return x.sum()

X = torch.rand(2, 20)
net = FancyMLP()
print(net)
net(X)

In [ ]:
"""
怎么获取模型内部的参数并修改。
"""
net = nn.Sequential(
    nn.Linear(4, 3), 
    nn.ReLU(), 
    nn.Linear(3, 1)
)  # pytorch已进行默认初始化

for name, param in net.named_parameters():
    print(name, param.size())

for name, param in net[0].named_parameters():
    print(name, param.size(), type(param))

print(net)
X = torch.rand(2, 4)
Y = net(X).sum()

"""
可以取出net的参数，修改参数
"""
weight_0 = list(net[0].parameters())[0]
print(weight_0.data)
print(weight_0.grad)
Y.backward()
print(weight_0.grad)

for name, param in net.named_parameters():
    if 'weight' in name:
        print("before init")
        print(name, param.data)
        init.normal_(param, mean=0, std=0.01)
        print("after init")
        print(name, param.data)

In [ ]:
"""
共享模型参数。
有点像传入了一个ref。
"""
linear = nn.Linear(1, 1, bias=False)
net = nn.Sequential(linear, linear) 
print(net)
for name, param in net.named_parameters():
    init.constant_(param, val=3)
    print(name, param.data)
print(id(net[0]) == id(net[1]))
print(id(net[0].weight) == id(net[1].weight))

In [ ]:
torch.cuda.is_available() 